# Introduction
In this notebook I will simplify the english->swedish translation task I have been tackeling in previous notebooks by limiting my data to sentences with a limited structure. 

In the [PyTorch tutorial](https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html#sphx-glr-download-intermediate-seq2seq-translation-tutorial-py) that I have been inspired by they limit their task to sentences start with "i am", "she is", "we are" etc.

Let's see if we have enough data in the english->swedish data set to use such a simplification.

# Data
Data from: http://www.manythings.org/anki/. 

In [1]:
data_path = 'data/swe-eng/swe.txt'
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')

Read all sentences.

In [2]:
input_sentences, target_sentences = [], []
for line in lines:
    try:
        input_text, target_text, *_ = line.split('\t')
    except ValueError:
        print(line)
        
    input_sentences.append(input_text)
    target_sentences.append(target_text)

## Tokenize and Normalise
I want to word tokenize by using NLTK, and then convert to lower case.

In [3]:
from nltk.tokenize import word_tokenize

In [4]:
input_tokenized = [list(map(str.lower, word_tokenize(sentence))) for sentence in input_sentences]
target_tokenized = [list(map(str.lower, word_tokenize(sentence, language='swedish'))) for sentence in target_sentences]

In [5]:
input_tokenized[0]

['run', '!']

## Limit sentence structure
Let's try the same limitations used in the PyTorch tutorial and see if we get enough data.

In [6]:
word_tokenize("I'm")

['I', "'m"]

In [7]:
eng_prefixes = [
    ["i",  "am"],
    ["i", "'m"],
    ["he", "is"],
    ["he", "'s"],
    ["she", "is"],
    ["she", "'s"],
    ["you", "are"],
    ["you" "'re"],
    ["we", "are"], 
    ["we", "'re"],
    ["they", "are"],
    ["they","'re"]
]

In [8]:
def matches_prefix(sentence, prefixes):
    for prefix in prefixes:
        if prefix == sentence[:len(prefix)]:
            return True
    return False

In [10]:
import numpy as np

In [11]:
match_idx = np.where(np.array([matches_prefix(sentence, eng_prefixes) for sentence in input_tokenized]))

In [12]:
print("%d sentences matching our prefix list" % len(match_idx[0]))

1073 sentences matching our prefix list


Not that many matching sentences, but let's attempt this limited problem. At least training will be faster.

In [32]:
input_sentences = np.array(input_tokenized)[match_idx]
target_sentences = np.array(target_tokenized)[match_idx]

## Limit sentence length

In [33]:
input_seq_lens = np.array([len(sentence) for sentence in input_sentences])
target_seq_lens = np.array([len(sentence) for sentence in target_sentences])

In [34]:
max_seq_len = 10

In [35]:
input_idx = np.where(input_seq_lens <= max_seq_len)
target_idx = np.where(target_seq_lens <= max_seq_len)

In [36]:
print("{} input sentences with {} or fewer characters".format(len(input_idx[0]), max_seq_len))
print("{} target sentences with {} or fewer characters".format(len(target_idx[0]), max_seq_len))

1032 input sentences with 10 or fewer characters
1042 target sentences with 10 or fewer characters


In [37]:
keep_idx = np.intersect1d(input_idx, target_idx)

In [38]:
print("{} input sentence pairs with {} or fewer characters in both languages".format(len(keep_idx), max_seq_len))

1025 input sentence pairs with 10 or fewer characters in both languages


I barely lose any data by limiting to 10 words per sentence.

In [39]:
input_sentences = np.array(input_sentences)[keep_idx]
target_sentences = np.array(target_sentences)[keep_idx]

## Build vocabularies
I'll pretty much copy the approach of the PyTorch tutorial to construct my vocabularies. Just doing some small code tweaks.

In [41]:
class Vocab:
    
    def __init__(self, name):
        self.name = name
        self.word2index = {"<SOS>" : 0, "<EOS>" : 1}
        self.word2count = {}
        # Reverse word2index
        self.index2word = dict([(b, a) for a, b in self.word2index.items()])
        self.n_words = len(self.word2index)
        
    def add_sentence(self, sentence):
        for word in sentence:
            self.add_word(word)
        
    def add_word(self, word):
        # If there any many more tokens than unique tokens, 
        # it is more efficient to assume the token is already in the vocabulary
        try:
            self.word2count[word] += 1
        except KeyError:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1    

In [42]:
input_vocab = Vocab('eng')
target_vocab = Vocab('swe')

In [43]:
%%time
for input_sentence, target_sentence in zip(input_sentences, target_sentences):
    input_vocab.add_sentence(input_sentence)
    target_vocab.add_sentence(target_sentence)

Wall time: 12 ms


In [44]:
for vocab in [input_vocab, target_vocab]:
    print("Vocab %s: %d unique tokens and %d tokens total" % (vocab.name, vocab.n_words, sum(vocab.word2count.values())))

Vocab eng: 845 unique tokens and 6337 tokens total
Vocab swe: 992 unique tokens and 5982 tokens total


## Divide data into a training and a validation set
I will use 800 sentances as training set and the remaining 225 as validation set.

In [45]:
trainig_size, validation_size = 800, len(input_sentences)-800

In [46]:
shuffle_idx = np.random.permutation(len(input_sentences))

In [47]:
train_idx, val_idx = shuffle_idx[:trainig_size], shuffle_idx[trainig_size:trainig_size+validation_size]

In [48]:
input_train, input_val = input_sentences[train_idx], input_sentences[val_idx]
target_train, target_val = target_sentences[train_idx], target_sentences[val_idx]

## Convert data to tensors
Just like in the PyTorch tutorial I will set up functions to convert the sentences to tensors.

In [49]:
import torch

In [50]:
def sentence2indexes(vocab, sentence):
    return [vocab.word2index[word] for word in sentence]


def sentence2tensor(vocab, sentence):
    indexes = sentence2indexes(vocab, sentence)
    indexes.append(vocab.word2index['<EOS>'])
    return torch.tensor(indexes, dtype=torch.long).view(-1, 1)


def pair2tensor(pair):
    input_tensor = sentence2tensor(input_vocab, pair[0])
    target_tensor = sentence2tensor(target_vocab, pair[1])
    return (input_tensor, target_tensor)

# Model
Let's use the same model as in my last notebook.

## Encoder

In [51]:
import torch.nn as nn

In [52]:
class EncoderRNN(nn.Module):
    def __init__(self, vocab_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        
        self.embedding = nn.Embedding(vocab_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        
    def forward(self, input_word, hidden_state):
        embedded = self.embedding(input_word).view(1, 1, -1)
        output, hidden_state = self.gru(embedded, hidden_state)
        return output, hidden_state
    
    # Returns a tensor with only zeroes to be used as initial hidden state
    def init_hidden(self):
        return torch.zeros(1, 1, self.hidden_size)
        

## Decoder

In [53]:
import torch.nn.functional as F

In [54]:
class DecoderRNNAttention(nn.Module):
    def __init__(self, vocab_size, hidden_size, dropout_p, max_length):
        super(DecoderRNNAttention, self).__init__()
        self.vocab_size = vocab_size
        self.hidden_size = hidden_size
        self.dropout_p = dropout_p
        self.max_length = max_length
        
        self.embedding = nn.Embedding(vocab_size, hidden_size)
        self.dropout = nn.Dropout(dropout_p)
        self.attention = nn.Linear(hidden_size * 2, max_length)
        self.attention_combine = nn.Linear(hidden_size * 2, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.output = nn.Linear(hidden_size, vocab_size)
        
    def forward(self, input_word, hidden_state, encoder_outputs):
        
        embedded = self.embedding(input_word).view(1, 1, -1)
        embedded = self.dropout(embedded)
        
        attention_weights = F.softmax(
            self.attention(torch.cat((embedded[0], hidden_state[0]), dim = 1)),
            dim = 1
        )
        
        attention_applied = torch.bmm(attention_weights.unsqueeze(0), encoder_outputs.unsqueeze(0))
        
        output = torch.cat((embedded[0], attention_applied[0]), dim=1)
        output = self.attention_combine(output).unsqueeze(0)
        
        output = F.relu(output)
        output, hidden_state = self.gru(output, hidden_state)
        
        output = F.log_softmax(self.output(output[0]), dim=1)
        return output, hidden_state, attention_weights
    
    # Returns a tensor with only zeroes to be used as initial hidden state
    def init_hidden(self):
        return torch.zeros(1, 1, self.hidden_size) 
        
        

# Training
The way I set up the model it can only train on one sentance at a time. This will probably add significant overhead, but tackeling that is a task for another day.

As in all my previous notebooks I will use teacher forcing to train. However, this time I am able to introduce a new variant! As suggested in the PyTorch tutorial it's possible to alternate between teacher forcing and feeding the previous output as input.

In [55]:
import random

In [56]:
def train_model(encoder, decoder, input_tensor, target_tensor, encoder_optimizer, 
                decoder_optimizer, criterion, max_length, teacher_forcing_ratio = 1.0):
    
    encoder_hidden_state = encoder.init_hidden()
    
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    
    input_length = input_tensor.shape[0]
    target_length = target_tensor.shape[0]
    
    encoder_outputs = torch.zeros(max_length, encoder.hidden_size)
    
    loss = 0
    
    # Encode input tensor
    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden_state)
        encoder_outputs[ei] = encoder_output
        
    # Decode input tensor, seeding the decoding with the <BOS> token
    decoder_input = torch.tensor([[target_vocab.word2index['<SOS>']]])
    decoder_hidden_state = encoder_hidden_state
    
    # Use teacher-forcing according to dice roll
    use_teacher_forcing = random.random() < teacher_forcing_ratio
    
    if use_teacher_forcing:
        for di in range(target_length):
            # Forward step and loss
            decoder_output, decoer_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden_state, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            
            # Update input for next step (teacher-forcing)
            decoder_input = target_tensor[di]
    else:      
        for di in range(target_length):
            # Forward step and loss
            decoder_output, decoer_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden_state, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            
            # Update input for next step (using output at this step)
            log_likelihood, top_word = decoder_output.topk(1) 
            decoder_input = top_word.squeeze().detach()
            
            if decoder_input.item() == target_vocab.word2index['<EOS>']:
                break
    
    loss.backward()
    encoder_optimizer.step()
    decoder_optimizer.step()
    
    return loss.item() / target_length
    
    

In [57]:
def tensor2sentence(tensor, vocab):
    sent = []
    for i in tensor:
        sent.append(vocab.index2word[i.item()])
    return sent

In [58]:
def evaluate_model(encoder, decoder, input_tensor, target_tensor, max_length):
    
    with torch.no_grad():

        encoder_hidden_state = encoder.init_hidden()

        input_length = input_tensor.shape[0]
        target_length = target_tensor.shape[0]

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size)

        # Encode input tensor
        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden_state)
            encoder_outputs[ei] = encoder_output

        # Decode input tensor, seeding the decoding with the <BOS> token
        decoder_input = torch.tensor([[target_vocab.word2index['<SOS>']]])
        decoder_hidden_state = encoder_hidden_state
        decoder_word_outputs = []

        for di in range(target_length):
            # Forward step and loss
            decoder_output, decoer_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden_state, encoder_outputs)

            # Update input for next step (using output at this step)
            log_likelihood, top_word = decoder_output.topk(1) 
            decoder_input = top_word.squeeze().detach()
            decoder_word_output = target_vocab.index2word[decoder_input.item()]
            decoder_word_outputs.append(decoder_word_output)
            if decoder_word_output == '<EOS>':
                break
                

        print("Input: ", " ".join(tensor2sentence(input_tensor, input_vocab)))
        print("Target: ", " ".join(tensor2sentence(target_tensor, target_vocab)))
        print("Result: ", " ".join(decoder_word_outputs))

    

In [59]:
encoder = EncoderRNN(input_vocab.n_words, 100)
decoder = DecoderRNNAttention(target_vocab.n_words, 100, .1, max_seq_len+1)

In [60]:
encoder_optimizer = torch.optim.Adam(encoder.parameters())
decoder_optimizer = torch.optim.Adam(decoder.parameters())

criterion = nn.NLLLoss()

In [61]:
from tqdm import tqdm

In [62]:
def train_iteratively(encoder, decoder, tensor_pairs, encoder_optimizer, 
                    decoder_optimizer, criterion, max_length, validation_pairs,
                    teacher_forcing_ratio = 1.0, epochs = 2):
    
    for epoch in range(epochs):
        print("Epoch %d / %d" % (epoch + 1, epochs))
        loss = 0
        for input_tensor, target_tensor in tqdm(tensor_pairs):
            
            loss += train_model(encoder, decoder, input_tensor, target_tensor, encoder_optimizer, 
                         decoder_optimizer, criterion, max_length, teacher_forcing_ratio)
        
        
        # Print some statistics every epoch
        epoch_loss = loss / len(tensor_pairs)
        print('Loss: {:.4f}'.format(epoch_loss))
        
        # Print some examples after each epoch
        # TODO: Don't just print the first 10 sentences of the validation set...
        for input_tensor, target_tensor in validation_pairs[:10]:
            evaluate_model(encoder, decoder, input_tensor, target_tensor, max_length)
            print("")
        print("")
        

In [69]:
def evaluate(encoder, decoder, tensor_pairs,  max_length):
    

    for input_tensor, target_tensor in tensor_pairs:
        evaluate_model(encoder, decoder, input_tensor, target_tensor, max_length)
        print("")

In [64]:
training_sentences = zip(input_train, target_train)
validation_sentences = zip(input_val, target_val)

training_tensors = [pair2tensor(sent_pair) for sent_pair in training_sentences]
validation_tensors = [pair2tensor(sent_pair) for sent_pair in validation_sentences]

My training setup could definitely be improved, but I can't spend to mucg time per coding session writing boilerplate. 
I'll improve it iteratively over coming projects!

Let's launch training!

In [65]:
train_iteratively(encoder, decoder, training_tensors, encoder_optimizer, decoder_optimizer, criterion, max_seq_len+1, validation_tensors)

Epoch 1 / 2


100%|████████████████████████████████████████████████████████████████████████████████| 800/800 [00:30<00:00, 26.52it/s]


Loss: 3.4339
Input:  i 'm late . <EOS>
Target:  jag är sen . <EOS>
Result:  jag är på . <EOS>

Input:  he is handsome . <EOS>
Target:  han är stilig . <EOS>
Result:  han är på . <EOS>

Input:  they 're stalling . <EOS>
Target:  de slingrar sig . <EOS>
Result:  de är på . <EOS>

Input:  i 'm from kyoto . <EOS>
Target:  jag är från kyoto . <EOS>
Result:  jag är på . <EOS>

Input:  he is always throwing his weight around . <EOS>
Target:  han håller jämt på och styr och ställer . <EOS>
Result:  han är på . <EOS>

Input:  he is suffering from a serious illness . <EOS>
Target:  han lider av en svår sjukdom . <EOS>
Result:  han är inte . <EOS>

Input:  he is always complaining about his low salary . <EOS>
Target:  han klagar jämt på sin låga lön . <EOS>
Result:  han är på . <EOS>

Input:  we 're not getting any younger . <EOS>
Target:  vi blir inte yngre . <EOS>
Result:  de är inte . <EOS>

Input:  she is my girlfriend . <EOS>
Target:  han är min vän . <EOS>
Result:  han är inte . <EOS>

Inpu

100%|████████████████████████████████████████████████████████████████████████████████| 800/800 [00:31<00:00, 25.59it/s]


Loss: 2.6232
Input:  i 'm late . <EOS>
Target:  jag är sen . <EOS>
Result:  jag är på att du

Input:  he is handsome . <EOS>
Target:  han är stilig . <EOS>
Result:  han är tystlåten . <EOS>

Input:  they 're stalling . <EOS>
Target:  de slingrar sig . <EOS>
Result:  de är på övervåningen .

Input:  i 'm from kyoto . <EOS>
Target:  jag är från kyoto . <EOS>
Result:  jag är på att du är

Input:  he is always throwing his weight around . <EOS>
Target:  han håller jämt på och styr och ställer . <EOS>
Result:  han är på övervåningen . <EOS>

Input:  he is suffering from a serious illness . <EOS>
Target:  han lider av en svår sjukdom . <EOS>
Result:  han är lite . <EOS>

Input:  he is always complaining about his low salary . <EOS>
Target:  han klagar jämt på sin låga lön . <EOS>
Result:  han är på övervåningen . <EOS>

Input:  we 're not getting any younger . <EOS>
Target:  vi blir inte yngre . <EOS>
Result:  de är inte så inte död

Input:  she is my girlfriend . <EOS>
Target:  han är min v

In [66]:
train_iteratively(encoder, decoder, training_tensors, encoder_optimizer, decoder_optimizer, criterion, max_seq_len+1, validation_tensors, epochs=5)

Epoch 1 / 5


100%|████████████████████████████████████████████████████████████████████████████████| 800/800 [00:34<00:00, 23.29it/s]


Loss: 2.2548
Input:  i 'm late . <EOS>
Target:  jag är sen . <EOS>
Result:  jag är från att .

Input:  he is handsome . <EOS>
Target:  han är stilig . <EOS>
Result:  han är ung . <EOS>

Input:  they 're stalling . <EOS>
Target:  de slingrar sig . <EOS>
Result:  de är gräsliga . <EOS>

Input:  i 'm from kyoto . <EOS>
Target:  jag är från kyoto . <EOS>
Result:  jag är från kanada . <EOS>

Input:  he is always throwing his weight around . <EOS>
Target:  han håller jämt på och styr och ställer . <EOS>
Result:  han är alltid på polisstationen . <EOS>

Input:  he is suffering from a serious illness . <EOS>
Target:  han lider av en svår sjukdom . <EOS>
Result:  han är ung . <EOS>

Input:  he is always complaining about his low salary . <EOS>
Target:  han klagar jämt på sin låga lön . <EOS>
Result:  han är alltid på övervåningen . <EOS>

Input:  we 're not getting any younger . <EOS>
Target:  vi blir inte yngre . <EOS>
Result:  de är inte död inte död

Input:  she is my girlfriend . <EOS>
Targ

100%|████████████████████████████████████████████████████████████████████████████████| 800/800 [00:36<00:00, 22.18it/s]


Loss: 1.9425
Input:  i 'm late . <EOS>
Target:  jag är sen . <EOS>
Result:  jag är på . <EOS>

Input:  he is handsome . <EOS>
Target:  han är stilig . <EOS>
Result:  han är ung . <EOS>

Input:  they 're stalling . <EOS>
Target:  de slingrar sig . <EOS>
Result:  de är barfota . <EOS>

Input:  i 'm from kyoto . <EOS>
Target:  jag är från kyoto . <EOS>
Result:  jag är från kanada . <EOS>

Input:  he is always throwing his weight around . <EOS>
Target:  han håller jämt på och styr och ställer . <EOS>
Result:  han är alltid bort han är alltid bort fel buss

Input:  he is suffering from a serious illness . <EOS>
Target:  han lider av en svår sjukdom . <EOS>
Result:  han är ung dit . <EOS>

Input:  he is always complaining about his low salary . <EOS>
Target:  han klagar jämt på sin låga lön . <EOS>
Result:  han är alltid bort . <EOS>

Input:  we 're not getting any younger . <EOS>
Target:  vi blir inte yngre . <EOS>
Result:  vi är inte död . <EOS>

Input:  she is my girlfriend . <EOS>
Target

...

Loss: 1.6537
Input:  i 'm late . <EOS>
Target:  jag är sen . <EOS>
Result:  jag är sen . <EOS>

Input:  he is handsome . <EOS>
Target:  han är stilig . <EOS>
Result:  han är ung . <EOS>

Input:  they 're stalling . <EOS>
Target:  de slingrar sig . <EOS>
Result:  de är barfota . <EOS>

Input:  i 'm from kyoto . <EOS>
Target:  jag är från kyoto . <EOS>
Result:  jag är från kanada . <EOS>

Input:  he is always throwing his weight around . <EOS>
Target:  han håller jämt på och styr och ställer . <EOS>
Result:  han är alltid bort mot fram emot att få höra

Input:  he is suffering from a serious illness . <EOS>
Target:  han lider av en svår sjukdom . <EOS>
Result:  han är ung . <EOS>

Input:  he is always complaining about his low salary . <EOS>
Target:  han klagar jämt på sin låga lön . <EOS>
Result:  han är alltid på scenen . <EOS>

Input:  we 're not getting any younger . <EOS>
Target:  vi blir inte yngre . <EOS>
Result:  vi är inte död . <EOS>

Input:  she is my girlfriend . <EOS>
Target

100%|████████████████████████████████████████████████████████████████████████████████| 800/800 [00:36<00:00, 21.88it/s]


Loss: 1.4145
Input:  i 'm late . <EOS>
Target:  jag är sen . <EOS>
Result:  jag är sen . <EOS>

Input:  he is handsome . <EOS>
Target:  han är stilig . <EOS>
Result:  han är ung . <EOS>

Input:  they 're stalling . <EOS>
Target:  de slingrar sig . <EOS>
Result:  de är nykter . <EOS>

Input:  i 'm from kyoto . <EOS>
Target:  jag är från kyoto . <EOS>
Result:  jag är från kanada på humör

Input:  he is always throwing his weight around . <EOS>
Target:  han håller jämt på och styr och ställer . <EOS>
Result:  han är alltid ivrig . <EOS>

Input:  he is suffering from a serious illness . <EOS>
Target:  han lider av en svår sjukdom . <EOS>
Result:  han är alltid på sjukhuset . <EOS>

Input:  he is always complaining about his low salary . <EOS>
Target:  han klagar jämt på sin låga lön . <EOS>
Result:  han är alltid bort . <EOS>

Input:  we 're not getting any younger . <EOS>
Target:  vi blir inte yngre . <EOS>
Result:  vi är inte död , .

Input:  she is my girlfriend . <EOS>
Target:  han är 

100%|████████████████████████████████████████████████████████████████████████████████| 800/800 [00:33<00:00, 23.61it/s]


Loss: 1.2163
Input:  i 'm late . <EOS>
Target:  jag är sen . <EOS>
Result:  jag är sen . <EOS>

Input:  he is handsome . <EOS>
Target:  han är stilig . <EOS>
Result:  han är nyfiken . <EOS>

Input:  they 're stalling . <EOS>
Target:  de slingrar sig . <EOS>
Result:  de är österrikare . <EOS>

Input:  i 'm from kyoto . <EOS>
Target:  jag är från kyoto . <EOS>
Result:  jag är från kanada på humör

Input:  he is always throwing his weight around . <EOS>
Target:  han håller jämt på och styr och ställer . <EOS>
Result:  han är alltid punktlig som dag . <EOS>

Input:  he is suffering from a serious illness . <EOS>
Target:  han lider av en svår sjukdom . <EOS>
Result:  han är alltid ivrig . <EOS>

Input:  he is always complaining about his low salary . <EOS>
Target:  han klagar jämt på sin låga lön . <EOS>
Result:  han är alltid ivrig . <EOS>

Input:  we 're not getting any younger . <EOS>
Target:  vi blir inte yngre . <EOS>
Result:  vi är inte varit . <EOS>

Input:  she is my girlfriend . <E

In [67]:
train_iteratively(encoder, decoder, training_tensors, encoder_optimizer, decoder_optimizer, criterion, max_seq_len+1, validation_tensors, epochs=5)

Epoch 1 / 5


100%|████████████████████████████████████████████████████████████████████████████████| 800/800 [00:31<00:00, 25.01it/s]


Loss: 1.0607
Input:  i 'm late . <EOS>
Target:  jag är sen . <EOS>
Result:  jag är sen . <EOS>

Input:  he is handsome . <EOS>
Target:  han är stilig . <EOS>
Result:  han är dit . <EOS>

Input:  they 're stalling . <EOS>
Target:  de slingrar sig . <EOS>
Result:  de är nykter . <EOS>

Input:  i 'm from kyoto . <EOS>
Target:  jag är från kyoto . <EOS>
Result:  jag är från humör till .

Input:  he is always throwing his weight around . <EOS>
Target:  han håller jämt på och styr och ställer . <EOS>
Result:  han är alltid punktlig eller hur ? <EOS>

Input:  he is suffering from a serious illness . <EOS>
Target:  han lider av en svår sjukdom . <EOS>
Result:  han är alltid tillbaka dit . <EOS>

Input:  he is always complaining about his low salary . <EOS>
Target:  han klagar jämt på sin låga lön . <EOS>
Result:  han är alltid på scenen . <EOS>

Input:  we 're not getting any younger . <EOS>
Target:  vi blir inte yngre . <EOS>
Result:  vi är inte varit . <EOS>

Input:  she is my girlfriend . <

100%|████████████████████████████████████████████████████████████████████████████████| 800/800 [00:31<00:00, 25.48it/s]


Loss: 0.9144
Input:  i 'm late . <EOS>
Target:  jag är sen . <EOS>
Result:  jag är sen . <EOS>

Input:  he is handsome . <EOS>
Target:  han är stilig . <EOS>
Result:  han är dit . <EOS>

Input:  they 're stalling . <EOS>
Target:  de slingrar sig . <EOS>
Result:  de är österrikare . <EOS>

Input:  i 'm from kyoto . <EOS>
Target:  jag är från kyoto . <EOS>
Result:  jag är från humör just .

Input:  he is always throwing his weight around . <EOS>
Target:  han håller jämt på och styr och ställer . <EOS>
Result:  han är alltid punktlig . <EOS>

Input:  he is suffering from a serious illness . <EOS>
Target:  han lider av en svår sjukdom . <EOS>
Result:  han är ung alltid ivrig att dö .

Input:  he is always complaining about his low salary . <EOS>
Target:  han klagar jämt på sin låga lön . <EOS>
Result:  han är alltid ivrig . <EOS>

Input:  we 're not getting any younger . <EOS>
Target:  vi blir inte yngre . <EOS>
Result:  vi är inte varit . <EOS>

Input:  she is my girlfriend . <EOS>
Target

...

Loss: 0.8039
Input:  i 'm late . <EOS>
Target:  jag är sen . <EOS>
Result:  jag är sen . <EOS>

Input:  he is handsome . <EOS>
Target:  han är stilig . <EOS>
Result:  han är nyfiken . <EOS>

Input:  they 're stalling . <EOS>
Target:  de slingrar sig . <EOS>
Result:  de är nog . <EOS>

Input:  i 'm from kyoto . <EOS>
Target:  jag är från kyoto . <EOS>
Result:  jag är från kanada . <EOS>

Input:  he is always throwing his weight around . <EOS>
Target:  han håller jämt på och styr och ställer . <EOS>
Result:  han är alltid punktlig eller hur samma sak människor .

Input:  he is suffering from a serious illness . <EOS>
Target:  han lider av en svår sjukdom . <EOS>
Result:  han är alltid på dåligt barn . <EOS>

Input:  he is always complaining about his low salary . <EOS>
Target:  han klagar jämt på sin låga lön . <EOS>
Result:  han är alltid på scenen . <EOS>

Input:  we 're not getting any younger . <EOS>
Target:  vi blir inte yngre . <EOS>
Result:  vi är inte varit . <EOS>

Input:  she i

100%|████████████████████████████████████████████████████████████████████████████████| 800/800 [00:34<00:00, 23.44it/s]


Loss: 0.7132
Input:  i 'm late . <EOS>
Target:  jag är sen . <EOS>
Result:  jag är sen . <EOS>

Input:  he is handsome . <EOS>
Target:  han är stilig . <EOS>
Result:  han är nyfiken . <EOS>

Input:  they 're stalling . <EOS>
Target:  de slingrar sig . <EOS>
Result:  de är nog . <EOS>

Input:  i 'm from kyoto . <EOS>
Target:  jag är från kyoto . <EOS>
Result:  jag är från turkiet . <EOS>

Input:  he is always throwing his weight around . <EOS>
Target:  han håller jämt på och styr och ställer . <EOS>
Result:  han är alltid punktlig sitt paraply inne rikaste man fel

Input:  he is suffering from a serious illness . <EOS>
Target:  han lider av en svår sjukdom . <EOS>
Result:  han är alltid ivrig . <EOS>

Input:  he is always complaining about his low salary . <EOS>
Target:  han klagar jämt på sin låga lön . <EOS>
Result:  han är alltid ivrig . <EOS>

Input:  we 're not getting any younger . <EOS>
Target:  vi blir inte yngre . <EOS>
Result:  vi tänker inte yngre än så

Input:  she is my gir

100%|████████████████████████████████████████████████████████████████████████████████| 800/800 [00:39<00:00, 20.09it/s]


Loss: 0.6295
Input:  i 'm late . <EOS>
Target:  jag är sen . <EOS>
Result:  jag är sen . <EOS>

Input:  he is handsome . <EOS>
Target:  han är stilig . <EOS>
Result:  han är dit . <EOS>

Input:  they 're stalling . <EOS>
Target:  de slingrar sig . <EOS>
Result:  de är banta . <EOS>

Input:  i 'm from kyoto . <EOS>
Target:  jag är från kyoto . <EOS>
Result:  jag är från turkiet . <EOS>

Input:  he is always throwing his weight around . <EOS>
Target:  han håller jämt på och styr och ställer . <EOS>
Result:  han är alltid punktlig som dag man fel buss .

Input:  he is suffering from a serious illness . <EOS>
Target:  han lider av en svår sjukdom . <EOS>
Result:  han är alltid tillbaka imorgon . <EOS>

Input:  he is always complaining about his low salary . <EOS>
Target:  han klagar jämt på sin låga lön . <EOS>
Result:  han är alltid på scenen . <EOS>

Input:  we 're not getting any younger . <EOS>
Target:  vi blir inte yngre . <EOS>
Result:  vi är inte yngre än så

Input:  she is my girlf

This time around I actually see some succesfull translations. However, I think the model is quickly overfitting to th training data, without improving on the validation set. 

I should have computed the validation loss to easier verify this...

Let's print some more examples, as the 10 I have chosen arbitrarily are not all that interesting.

In [70]:
evaluate(encoder, decoder, validation_tensors, max_seq_len+1)

Input:  i 'm late . <EOS>
Target:  jag är sen . <EOS>
Result:  jag är sen . <EOS>

Input:  he is handsome . <EOS>
Target:  han är stilig . <EOS>
Result:  han är dit . <EOS>

Input:  they 're stalling . <EOS>
Target:  de slingrar sig . <EOS>
Result:  de är banta . <EOS>

Input:  i 'm from kyoto . <EOS>
Target:  jag är från kyoto . <EOS>
Result:  jag är från kanada . <EOS>

Input:  he is always throwing his weight around . <EOS>
Target:  han håller jämt på och styr och ställer . <EOS>
Result:  han är alltid punktlig så söt eller tokig . <EOS>

Input:  he is suffering from a serious illness . <EOS>
Target:  han lider av en svår sjukdom . <EOS>
Result:  han är alltid från turkiet . <EOS>

Input:  he is always complaining about his low salary . <EOS>
Target:  han klagar jämt på sin låga lön . <EOS>
Result:  han är alltid på scenen . <EOS>

Input:  we 're not getting any younger . <EOS>
Target:  vi blir inte yngre . <EOS>
Result:  vi är inte yngre än .

Input:  she is my girlfriend . <EOS>
T

Target:  jag är arg på dig . <EOS>
Result:  jag är här för er . <EOS>

Input:  they are my grandfather 's books . <EOS>
Target:  de är min morfars böcker . <EOS>
Result:  de är min farfars böcker . <EOS>

Input:  i 'm glad tom has gone . <EOS>
Target:  jag är glad att tom är borta . <EOS>
Result:  jag är glad att tom . <EOS>

Input:  they 're not busy . <EOS>
Target:  de är inte upptagna . <EOS>
Result:  de är inte upptagen . <EOS>

Input:  he 's strong . <EOS>
Target:  han är stark . <EOS>
Result:  han är här . <EOS>

Input:  we are men . <EOS>
Target:  vi är män . <EOS>
Result:  vi är hungrig . <EOS>

Input:  i 'm so proud of you . <EOS>
Target:  jag är så stolt över dig . <EOS>
Result:  jag är så söt er allihop . <EOS>

Input:  he is always making a fool of me . <EOS>
Target:  han driver alltid med mig . <EOS>
Result:  han är alltid rätt alltid punktlig en

Input:  i 'm painting easter eggs . <EOS>
Target:  jag målar påskägg . <EOS>
Result:  jag kommer jämt dit .

Input:  i 'm not a

Input:  they are the ones who want to go . <EOS>
Target:  det är de som vill gå . <EOS>
Result:  de närmar sig . <EOS>

Input:  i 'm your boss now . <EOS>
Target:  jag är din chef nu . <EOS>
Result:  jag är din advokat . <EOS>

Input:  i 'm here every night . <EOS>
Target:  jag är här varje kväll . <EOS>
Result:  jag är här . <EOS>

Input:  he 's good at cards . <EOS>
Target:  han är duktig med kort . <EOS>
Result:  han är bra på skolan . <EOS>

Input:  i 'm not from a rich family . <EOS>
Target:  jag kommer inte från en rik familj . <EOS>
Result:  jag är inte härifrån . <EOS>

Input:  we 're all related . <EOS>
Target:  vi är alla släkt . <EOS>
Result:  vi är alla lika . <EOS>

Input:  i 'm glad we agree , tom . <EOS>
Target:  jag är glad att vi är överens . <EOS>
Result:  jag är här för detta glad att vi är

Input:  i 'm so full . <EOS>
Target:  jag är så mätt . <EOS>
Result:  jag är så söt . <EOS>

Input:  he 's the captain of a team . <EOS>
Target:  han är lagkapten . <EOS>
Result:

Target:  jag är inte beredd att dö än . <EOS>
Result:  jag är inte beredd att dö . <EOS>

Input:  i 'm glad we found you . <EOS>
Target:  jag är glad att vi hittade dig . <EOS>
Result:  jag är glad att vi träffades . <EOS>

Input:  i 'm waiting for you in my room . <EOS>
Target:  jag väntar på dig på mitt rum . <EOS>
Result:  jag tänker föräldrar . <EOS>

Input:  we 're different . <EOS>
Target:  vi är olika . <EOS>
Result:  vi är marys . <EOS>

Input:  i 'm here , too . <EOS>
Target:  jag är också här . <EOS>
Result:  jag är här . <EOS>

Input:  i 'm going to try . <EOS>
Target:  jag ska försöka . <EOS>
Result:  jag kommer att åka .

Input:  i 'm not prepared to compromise . <EOS>
Target:  jag är inte beredd att kompromissa . <EOS>
Result:  jag är inte ! inte det . <EOS>

Input:  he is always speaking ill of his wife . <EOS>
Target:  han talar alltid illa om sin fru . <EOS>
Result:  han är alltid dumma frågor . <EOS>

Input:  he 's a drama queen . <EOS>
Target:  han är en drama queen 

There are some examples of perfect, or close translations! However, the model consequently fails at longer sentences and less common words. I think it is seriously suffering from the small data set. How can it learn to represent rare words like locations etc?

So, I limited my data set to only include sentences with a limited structure, in this case they are only allowed to start with a few different word combinations. As far as I can see my model distinguishes well between the allowe starts! 
My problem is now instead that the model has trouble learning to translate the words beyond the start, as these can be fa more varied. 

Any translation model with a similar setup as mine will struggle with out of vocabulary words, since they never appear in the training data I would expect the model to never use them. 

A proper training set should be representative of the true data distribution. In a normal translation setting this data distribution should cover all english sentences and their swedish translation, in my fabriacted task it should be all english sentences starting with one of my chosen prefixes. Neither my original data set or my reduced set can get close to representing these distributions.

Perhaps it's hopeless to train a translation system on data of such limied size as I am doing? 

I have many ideas I would like to incorporate, such as character embeddings and more regularization, but with my current results it would be hard to meassure their effect.